In [7]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow.python'

In [6]:
!pip install --upgrade tensorflow

  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl (276.6 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\tahma\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\tahma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Read the databases from resources
health_factors = pd.read_csv("/Resources/Health_Risk_factors.csv")

In [ ]:
# Renaming columns in the health_factors DataFrame to more descriptive names
hf = health_factors.rename(columns={
    "Prevalence of smoking Male % of adults 2016": "smoking_male",
    "Prevalence of smoking female % of adults 2016": "smoking_female",
    "Incidence of tuberculosis  per 100,000 people 2018": "tuberculosis",
    "Prevalence of HIV Total % of population ages 15-49 2018": "HIV_total",
    "Prevalence of HIV Women's share of population ages 15+ living with HIV % 2018": "HIV_female",
    "Prevalence of HIV Youth, Male % of population ages 15-24 2018": "HIV_youth_male",
    "Prevalence of HIV Youth, Female % of population ages 15-24 2018": "HIV_youth_female",
    "Antiretroviral therapy coverage  % of people living with HIV 2018": "HIV_therapy",
    "Prevalence of diabetes  % of population ages 20 to 79 2019": "diabetes",
    "Cause of death Communicable diseases and maternal, prenatal, and nutrition conditions % of population 2016":"Diseases_death",
    "Cause of death Non-communicable diseases % of population 2016": "Chronic_death",
    "Cause of death Injuries % of population 2016": "injury_death"
})

hf.columns


In [8]:
df=hf[['HIV_total','HIV_female', 'HIV_youth_male','HIV_youth_female', 'Diseases_death', 'tuberculosis' ]]
df1 = df.replace('', np.nan)  # Replace empty values with NaN
df1 = df.dropna()  # Drop rows with any missing values


In [9]:
df1["tuberculosis"].describe()


count    137.000000
mean     125.824818
std      145.431048
min        0.000000
25%       18.000000
50%       68.000000
75%      182.000000
max      611.000000
Name: tuberculosis, dtype: float64

In [11]:
# Define the thresholds for classification
threshold_HIV_total = 1.5
threshold_HIV_female = 45
threshold_HIV_youth_male = 0.4
threshold_HIV_youth_female = 0.3
threshold_Diseases_death = 45
threshold_tuberculosis = 100
# Create a DataFrame to store the classified values
df2 = pd.DataFrame()
# Classify every columns
df2['HIV_total'] = [0 if i < threshold_HIV_total else 1 for i in df1['HIV_total']]
df2['HIV_female'] = [0 if i < threshold_HIV_female else 1 for i in df1['HIV_female']]
df2['HIV_youth_male'] = [0 if i < threshold_HIV_youth_male else 1 for i in df1['HIV_youth_male']]
df2['HIV_youth_female'] = [0 if i < threshold_HIV_youth_female else 1 for i in df1['HIV_youth_female']]
df2['Diseases_death'] = [0 if i < threshold_Diseases_death else 1 for i in df1['Diseases_death']]
df2['tuberculosis'] = [0 if i < threshold_tuberculosis else 1 for i in df1['tuberculosis']]
# Extract X and y
y = df2['tuberculosis'].values
X = df2.drop(columns='tuberculosis').values


In [12]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1,stratify=y,test_size=.2)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
l_1 = 50
l_2 = 50
input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=l_1, input_dim = input_features, activation = "tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=l_2, activation = "LeakyReLU"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                300       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2,901
Trainable params: 2,901
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
nn.fit(X_train, y_train, epochs=50)

Epoch 1/50
4/4 [==============================] - 2s 25ms/step - loss: 0.6784 - accuracy: 0.7615
Epoch 2/50
4/4 [==============================] - 0s 18ms/step - loss: 0.6407 - accuracy: 0.7982
Epoch 3/50
4/4 [==============================] - 0s 14ms/step - loss: 0.6181 - accuracy: 0.7982
Epoch 4/50
4/4 [==============================] - 0s 13ms/step - loss: 0.6022 - accuracy: 0.7982
Epoch 5/50
4/4 [==============================] - 0s 18ms/step - loss: 0.5917 - accuracy: 0.7982
Epoch 6/50
4/4 [==============================] - 0s 17ms/step - loss: 0.5798 - accuracy: 0.7982
Epoch 7/50
4/4 [==============================] - 0s 14ms/step - loss: 0.5686 - accuracy: 0.7982
Epoch 8/50
4/4 [==============================] - 0s 10ms/step - loss: 0.5551 - accuracy: 0.7982
Epoch 9/50
4/4 [==============================] - 0s 11ms/step - loss: 0.5429 - accuracy: 0.7982
Epoch 10/50
4/4 [==============================] - 0s 10ms/step - loss: 0.5304 - accuracy: 0.7982
Epoch 11/50
4/4 [============

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.6042 - accuracy: 0.7500 - 161ms/epoch - 161ms/step
Loss: 0.6042343974113464, Accuracy: 0.75


In [19]:
# Predict on the test data
y_pred = nn.predict(X_test)

# The predictions will be in the form of probabilities, so you might want to round them to get class labels (0 or 1)
y_pred_class = y_pred.round()
# Calculate the accuracy of the predictions
correct_predictions = (y_pred_class == y_test.reshape(-1, 1)).sum()
total_samples = len(y_test)
accuracy = correct_predictions / total_samples
print("Accuracy:", accuracy)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_class))

1/1 [==============================] - 0s 22ms/step
Accuracy: 0.75
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.88      0.81        17
           1       0.75      0.55      0.63        11

    accuracy                           0.75        28
   macro avg       0.75      0.71      0.72        28
weighted avg       0.75      0.75      0.74        28

